# Endproject GIS Analysis <br> Location Analysis for a new Stadium in Graz <br> (Paar, Flor, Wallisch)

In [ ]:
# Data manipulation
import pandas as pd
import numpy as np

# Geospatial
import geopandas as gpd
from shapely import wkt
from shapely.geometry import Polygon
import osmnx as ox
import networkx as nx

# Raster processing
import os
import pyproj
# optinal code fix with PROJ_LIB if pyproj doesnt work properly
os.environ["PROJ_LIB"] = pyproj.datadir.get_data_dir()

import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from pyproj import CRS
from rasterio.features import shapes
from shapely.geometry import shape

# Visualization
import keplergl
import matplotlib.pyplot as plt
import contextily as ctx
from matplotlib_scalebar.scalebar import ScaleBar



Data Preparation
===============================================================================     

Getting district boundaries of Graz (District boundaries (Overpass Turbo (n.d.). Overpass API web interface. https://overpass-turbo.eu/ (Accessed December 2, 2025))) <br>

In [ ]:
#loading GeoJson Districts of Graz
gdf_districts = gpd.read_file("data/district_graz.geojson")
gdf_districts = gdf_districts[["name", "geometry"]].copy() #extract the columns needed

# Exclusion Areas

Getting hard exklusion layers from OpenStreetMap via OSMnx, set the CRS to 32633 (UTM zone 33N) and buffer some of them:<br>
- water bodies  <br>
- parks/green areas <br>
- transport infrastructure <br>
- buildings

In [ ]:
# Define the place name for OSMnx
PLACE_NAME:str = "Graz, Austria"

In [ ]:
# Define tags for water features and load them using OSMnx
tags_water = {
    "natural": ["water"],
    "waterway": ["river", "stream", "canal", "ditch"],
    "landuse": ["reservoir"],
    "water": ["lake", "river", "pond", "basin"]
}
gdf_water = ox.features_from_place(
    PLACE_NAME,
    tags=tags_water
)
gdf_water = gdf_water[["geometry"]].copy()
gdf_water["category"] = "water"

# Reproject
gdf_water = gdf_water.to_crs(epsg=32633)

# 20 meter buffer
gdf_water_Buffer20m = gdf_water.copy()
gdf_water_Buffer20m["geometry"] = gdf_water_Buffer20m.buffer(20)
gdf_water_Buffer20m.head()

In [ ]:
# Define tags for parks and load them using OSMnx
tags_parks = {
    "leisure": [
        "park", "garden", "playground", "recreation_ground"
    ],
    "boundary": [
        "protected_area"
    ]
}
gdf_parks = ox.features_from_place(
    PLACE_NAME,
    tags=tags_parks
)

gdf_parks = gdf_parks[["geometry"]].copy()
gdf_parks["category"] = "green areas"

# Reproject
gdf_parks = gdf_parks.to_crs(epsg=32633)

# 20 meter buffer
gdf_parks_Buffer20m = gdf_parks.copy()
gdf_parks_Buffer20m["geometry"] = gdf_parks_Buffer20m.buffer(20)
gdf_parks_Buffer20m.head(5)

In [ ]:
# Define tags for transport infrastructure (road and rail network) and load them using OSMnx
tags_transport = {
    "highway": [
        "motorway", "trunk", "primary", "secondary", "tertiary",
        "motorway_link", "trunk_link", "primary_link", "secondary_link"
    ],
    "railway": [
        "rail", "tram", "light_rail", "subway"
    ]
}

gdf_transport = ox.features_from_place(
    PLACE_NAME,
    tags=tags_transport
)

gdf_transport = gdf_transport[["geometry"]].copy()
gdf_transport["category"] = "transport infrastructure"

# Reproject
gdf_transport = gdf_transport.to_crs(epsg=32633)

# 20 meter buffer
gdf_transport_Buffer20m = gdf_transport.copy()
gdf_transport_Buffer20m["geometry"] = gdf_transport_Buffer20m.buffer(20)
gdf_transport_Buffer20m.head(20)

In [ ]:
# Define tags for buildings and load them using OSMnx
tags_buildings_hard = {
    "building": True
}

gdf_buildings = ox.features_from_place(
    PLACE_NAME,
    tags=tags_buildings_hard
)

gdf_buildings = gdf_buildings[["geometry"]].copy()
gdf_buildings["category"] = "buildings_hard"

# Reproject and create 30 meter buffer (the building buffers may differ with the shape of a city and garden areas)
gdf_buildings = gdf_buildings.to_crs(epsg=32633)
gdf_buildings_Buffer30m = gdf_buildings.copy()
gdf_buildings_Buffer30m["geometry"] = gdf_buildings_Buffer30m.buffer(30)
gdf_buildings_Buffer30m.head(5)

In [ ]:
# Define tags for cemeteries and load them using OSMnx
tags_cemetery = {
    "landuse": ["cemetery"],
    "amenity": ["grave_yard"]
}

gdf_cemetery = ox.features_from_place(
    PLACE_NAME,
    tags=tags_cemetery
)

gdf_cemetery = gdf_cemetery[["geometry"]].copy()
gdf_cemetery["category"] = "cemetery"

gdf_cemetery = gdf_cemetery.to_crs(epsg=32633)
gdf_cemetery.head(5)

Merge to one geodataframe and dissolve to one geometry

In [ ]:
# Combine all exclusion areas into a single GeoDataFrame
gdf_exclusion_areas = pd.concat([
    gdf_water_Buffer20m,
    gdf_parks_Buffer20m,
    gdf_transport_Buffer20m,
    gdf_buildings_Buffer30m,
    gdf_cemetery,
], ignore_index=True)

In [ ]:
gdf_exclusion_areas.head(5)

In [ ]:
# Control the geometry 
gdf_exclusion_areas.geometry.geom_type.value_counts()


In [ ]:
# Remove point geometries and explode Multipolygons in single Polygons
gdf_exclusion_areas = gdf_exclusion_areas[
    gdf_exclusion_areas.geometry.geom_type != "Point"
]
gdf_exclusion_areas = gdf_exclusion_areas.explode(
    ignore_index=True
)
gdf_exclusion_areas.geometry.geom_type.value_counts()

Reprojecting DEM from WGS84 to UTM33N and calculate slope <br>
data source: <br>
	- DEM (OpenTopography (n.d.). Copernicus GLO-30 Digital Elevation Model. https://portal.opentopography.org/raster?opentopoID=OTSDEM.032021.4326.3
	(Accessed January 15th, 2025))

In [ ]:
dem_in = "data/DEM30_Graz.tif"          # original DEM
dem_utm = "data/DEM30_Graz_utm33.tif"   # output DEM (UTM33N)

# Reproject the DEM to UTM Zone 33N
with rasterio.open(dem_in) as src:
    src_crs = src.crs if src.crs is not None else CRS.from_epsg(4326)
    dst_crs = CRS.from_epsg(32633)  # UTM Zone 33N

    transform, width, height = calculate_default_transform(
        src_crs, dst_crs, src.width, src.height, *src.bounds
    )
    # create a new profile for the output DEM
    profile = src.profile.copy()
    profile.update({
        "crs": dst_crs,
        "transform": transform,
        "width": width,
        "height": height,
        "nodata": src.nodata if src.nodata is not None else -9999
    })
    # write the reprojected DEM to a new file
    with rasterio.open(dem_utm, "w", **profile) as dst: 
        dst_array = np.full((height, width), profile["nodata"], dtype=profile["dtype"])
        dst.write(dst_array, 1)

        reproject(
            source=rasterio.band(src, 1),
            destination=rasterio.band(dst, 1),
            src_transform=src.transform,
            src_crs=src_crs,
            dst_transform=transform,
            dst_crs=dst_crs,
            resampling=Resampling.bilinear,
            src_nodata=src.nodata,
            dst_nodata=profile["nodata"]
        )

In [ ]:
# Calculate slope from DEM

dem_path = "data/DEM30_Graz_utm33.tif"
slope_path = "data/dem_slope.tif"

with rasterio.open(dem_path) as src:
    dem = src.read(1, masked=True).astype("float64")
    profile = src.profile.copy()
    transform = src.transform

# Calculate the resolution in x and y directions
xres = transform.a
yres = abs(transform.e)

# Calculate the gradients in x and y directions
dzdy, dzdx = np.gradient(dem.filled(np.nan), yres, xres)

# Calculate the slope in radians and convert to degrees
slope_rad = np.arctan(np.sqrt(dzdx**2 + dzdy**2))
slope_deg = np.degrees(slope_rad)

nodata_out = -9999.0
slope_out = np.where(np.isnan(slope_deg), nodata_out, slope_deg).astype(np.float32)

profile.update(dtype=rasterio.float32, count=1, nodata=nodata_out)

with rasterio.open(slope_path, "w", **profile) as dst:
    dst.write(slope_out, 1)


Polygonize slope raster for areas with slope greater than threshold (e.g. 5 degrees)

In [ ]:
# Filter slope raster and convert to polygons

slope_path = "data/dem_slope.tif"
out_path   = "data/slope_filtered.shp"

threshold   = 5.0     # degrees of slope
min_area_m2 = 500.0   # minimum area of polygons to keep 

with rasterio.open(slope_path) as src:
    slope = src.read(1)
    transform = src.transform
    crs = src.crs
    nodata = src.nodata

# Create a mask for pixels where slope is greater than the threshold
mask = slope > threshold
if nodata is not None:
    mask = mask & (slope != nodata)
mask = mask & np.isfinite(slope)

value_raster = mask.astype(np.uint8)

# Extract polygons from the raster where value is 1 (slope > threshold)
geoms = []
for geom, value in shapes(value_raster, mask=mask, transform=transform):
    if value == 1:
        geoms.append(shape(geom))

gdf_slope = gpd.GeoDataFrame(geometry=geoms, crs=crs)

gdf_slope["diss"] = 1
gdf_slope = gdf_slope.dissolve(by="diss")
gdf_slope = gdf_slope.explode(index_parts=False).reset_index(drop=True)

gdf_slope["area_m2"] = gdf_slope.area
gdf_slope = gdf_slope[gdf_slope["area_m2"] >= min_area_m2]

# Save the resulting polygons to a ESRI shapefile
gdf_slope["slope_gt"] = threshold
gdf_slope = gdf_slope[["slope_gt", "area_m2", "geometry"]]
gdf_slope.to_file(out_path, driver="ESRI Shapefile")

print(f"{len(gdf_slope)} polygons saved to {out_path}")

In [ ]:
# Remove steep slope areas from Graz districts

graz = gdf_districts.copy()
steep = gpd.read_file("data/slope_filtered.shp")

if graz.crs != steep.crs:
    graz = graz.to_crs(steep.crs)

# fix potential geometry issues
graz["geometry"] = graz.geometry.buffer(0)
steep["geometry"] = steep.geometry.buffer(0)

#union all areas 
steep_union = steep.geometry.union_all()

# differnce
result = graz.copy()
result["geometry"] = result.geometry.difference(steep_union)

# remove empty geometries
result = result[result.geometry.notna() & ~result.is_empty]

result.to_file("data/graz_suitable_slope.shp", driver="ESRI Shapefile")



Final suitable areas are those slope areas that do not intersect with exclusion areas

In [ ]:
# Remove exclusion areas from slope suitable areas

suitable = gpd.read_file("data/graz_suitable_slope.shp")
excluded = gdf_exclusion_areas.copy()

if suitable.crs != excluded.crs:
    excluded = excluded.to_crs(suitable.crs)
    
# fix potential geometry issues
suitable["geometry"] = suitable.geometry.buffer(0)
excluded["geometry"] = excluded.geometry.buffer(0)

excluded_union = excluded.geometry.union_all()

suitable_area = suitable.copy()
suitable_area["geometry"] = suitable_area.geometry.difference(excluded_union)

suitable_area = suitable_area[suitable_area.geometry.notna() & ~suitable_area.is_empty].copy()

suitable_area = suitable_area.explode(index_parts=False).reset_index(drop=True)
suitable_area = suitable_area[suitable_area.geometry.notna() & ~suitable_area.is_empty].copy()

# calculate area and filter by minimum size (filtering for at least 13 ha)
suitable_area["area_m2"] = suitable_area.area
suitable_area = suitable_area[suitable_area["area_m2"] >= 130000][["area_m2", "geometry"]]

suitable_area.to_file("data/graz_final_suitable_areas_slope.shp", driver="ESRI Shapefile")
print("ready:", len(suitable_area), "Polygons")

In [ ]:
print("Graz CRS:", graz.crs)
print("Steep CRS:", steep.crs)
slope_graz = gpd.read_file("data/graz_suitable_slope.shp")
print("Slope CRS:", slope_graz.crs)

### Distance Calculation for nearest distances to relevant point and polygon layers

nearest distances to hospitals, cemetries, kindergardens, religious areas, and residential areas <br>
all distances are calculated in meters <br>
set your desired EPSG code (default: 32633) <br>
shorter distances are worse (higher distance values are better) <br>
(using function add_nearest_distances)

In [ ]:
# get relevant point and polygon layers from OSM via osmnx
gdf_kindergarten = ox.features_from_place(PLACE_NAME, tags={"amenity": ["kindergarten"]})
gdf_hospital = ox.features_from_place(PLACE_NAME, tags={"amenity": ["hospital"]})
gdf_religious = ox.features_from_place(PLACE_NAME, tags={"amenity": ["place_of_worship"]})
gdf_residential = ox.features_from_place(PLACE_NAME, tags={"landuse": ["residential"]})[["geometry"]]

In [ ]:
# function to convert geometries to points and set right crs
def only_points(gdf, category, epsg=32633):
    g = gdf[["geometry"]].to_crs(epsg).assign(category=category)
    g["geometry"] = g.geometry.representative_point().where(g.geom_type != "Point", g.geometry)
    return g[["category", "geometry"]]

In [ ]:
# function to calculate nearest distances
def add_nearest_distances(suitable_area, point_layers, polygon_layers=None, epsg=32633, prefix="dist_"):
    out = suitable_area.to_crs(epsg).copy()
    src = out[["geometry"]]

    # Point-Layers (all except residential)
    for cat, gdf in point_layers.items():
        targets = only_points(gdf, cat, epsg=epsg)[["geometry"]]
        out[f"{prefix}{cat}"] = gpd.sjoin_nearest(src, targets, how="left", distance_col="d")["d"].to_numpy()
        
    # Polygon-Layers (residential areas)
    if polygon_layers:
        for cat, gdf in polygon_layers.items():
            targets = gdf[["geometry"]].to_crs(epsg)
            out[f"{prefix}{cat}"] = gpd.sjoin_nearest(src, targets, how="left", distance_col="d")["d"].to_numpy()

    return out


In [ ]:
# set dictionaries for point and polygon layers and 
# calculate nearest distances via function (add_nearest_distances)
point_layers = {
    "kindergarten": gdf_kindergarten,
    "hospital": gdf_hospital,
    "religious": gdf_religious,
    "cemetries": gdf_cemetery,
}

polygon_layers = {
    "residential": gdf_residential,
}

suitable_area = add_nearest_distances(
    suitable_area,
    point_layers=point_layers,
    polygon_layers=polygon_layers,
    epsg=32633,
    prefix="dist_"
)
print(suitable_area.head())

In [ ]:
# check if suitable areas overlap with residential areas
# when they overlap the distance is Zero and BOOLEAN true
res_union = gdf_residential.to_crs(suitable_area.crs).geometry.union_all()
print(suitable_area.geometry.intersects(res_union))


### Distance Calculation for cars and public transport access

Calculation of walking distance from suitable area to Public transport stops <br>
Calculation of driving distance from suitable area to highway exits <br>
using multisource dijkstra algorithm (add_nearest_network_distance)

In [ ]:
# Drivable network (for car accessibility)
drive_n = ox.graph_from_place(PLACE_NAME, network_type="drive")
drive_n = ox.project_graph(drive_n, to_crs="EPSG:32633")

# Walkable network (for pedestrian/public transport access)
walk_n = ox.graph_from_place(PLACE_NAME, network_type="walk")
walk_n = ox.project_graph(walk_n, to_crs="EPSG:32633")

In [ ]:
# motorway exits
gdf_exits = ox.features_from_place(PLACE_NAME, tags={"highway": ["motorway_junction"]})

# public transport stops
gdf_stops = ox.features_from_place(PLACE_NAME,tags={
    "highway": ["bus_stop"],
    "public_transport": ["platform", "stop_position"],
    "railway": ["tram_stop", "halt", "station"],})

In [ ]:
# function to get nearest nodes from gdf points
def gdf_to_nearest_nodes(gdf_points, G, epsg=32633, unique=True, category="_"):
    pts = only_points(gdf_points, category=category, epsg=epsg)
    xs = pts.geometry.x.to_numpy()
    ys = pts.geometry.y.to_numpy()
    nodes = ox.distance.nearest_nodes(G, X=xs, Y=ys)
    nodes = list(nodes)
    if unique:
        nodes = list(dict.fromkeys(nodes))
    return nodes

In [ ]:
# function to get nearest nodes from suitable polygons
def polygons_to_start_nodes(polygons_gdf, G, epsg=32633):
    poly = polygons_gdf.to_crs(epsg)
    src_pts = poly.geometry.representative_point()
    start_nodes = ox.distance.nearest_nodes(G, X=src_pts.x.to_numpy(), Y=src_pts.y.to_numpy())
    return list(start_nodes)

In [ ]:
#function to calculate nearest network distance from polygons to target nodes
def add_nearest_network_distance(polygons_gdf, G, target_nodes, epsg=32633, colname="netdist", weight="length"):
    out = polygons_gdf.to_crs(epsg).copy()

    # start nodes for each polygon
    start_nodes = polygons_to_start_nodes(out, G, epsg=epsg)

    # distance to nearest target for all nodes (one run)
    dist_map = nx.multi_source_dijkstra_path_length(G, sources=target_nodes, weight=weight)

    # assign result per polygon (NaN if unreachable)
    out[colname] = np.array([dist_map.get(n, np.nan) for n in start_nodes], dtype=float)
    return out

In [ ]:
# Walking network distances to public transport stops
stop_nodes = gdf_to_nearest_nodes(gdf_stops, walk_n, epsg=32633, unique=True)

suitable_area = add_nearest_network_distance(
    suitable_area,
    G=walk_n,
    target_nodes=stop_nodes,
    epsg=32633,
    colname="dist_stops",
    weight="length"
)
print(suitable_area.head())


In [ ]:
# Verify that areas with zero network distance to stops indeed contain a stop (TRUE when they do)
start_nodes = polygons_to_start_nodes(suitable_area, walk_n, epsg=32633)
stop_nodes = set(gdf_to_nearest_nodes(gdf_stops, walk_n, epsg=32633, unique=True, category="stops"))
print(suitable_area.loc[suitable_area["dist_stops"].eq(0)].index.tolist(),
      [n in stop_nodes for n in start_nodes])


In [ ]:
# driving network distances to highway exits
exit_nodes = gdf_to_nearest_nodes(gdf_exits, drive_n, epsg=32633, unique=True)

suitable_area = add_nearest_network_distance(
    suitable_area,
    G=drive_n,
    target_nodes=exit_nodes,
    epsg=32633,
    colname="dist_exits",
    weight="length"
)
print(suitable_area.head())

# Scoring and Weighting

In [ ]:
# scores when further away is better
def score_far(dist, min_dist, max_dist):
    return ((dist - min_dist) / (max_dist - min_dist)).clip(0, 1) # normalized 0-1

# score when nearer is better
def score_near(dist, min_dist, max_dist):
    return (1 - (dist - min_dist) / (max_dist - min_dist)).clip(0, 1)


In [ ]:
# Final scoring rules 
# distance rules when further away is better (FAR) and nearer is better (NEAR)

far_rules = {
    "dist_residential":  (0, 2500), #min, max to residential areas
    "dist_kindergarten": (0, 1500), #min, max to kindergartens
    "dist_religious":    (0, 1500),  #min, max to religious places
    "dist_hospital":     (0, 3000), #min, max to hospitals
    "dist_cemetries":    (0, 1500), #min, max to cemetries
}

near_rules = {
    "dist_stops": (0, 1500),  #min, max to public transport stops
    "dist_exits": (0, 6000),} #min, max to motorway exits

In [ ]:
# make new GeoDataFrame for scores and calculate individual scores
gdf_scores = suitable_area.copy()

# FAR scores
for col, (mn, mx) in far_rules.items():
    newcol = "score_" + col.replace("dist_", "")
    gdf_scores[newcol] = score_far(gdf_scores[col], mn, mx)

# NEAR scores
for col, (mn, mx) in near_rules.items():
    newcol = "score_" + col.replace("dist_", "")
    gdf_scores[newcol] = score_near(gdf_scores[col], mn, mx)


In [ ]:
# Weights for each criteria (when more important, higher weight factor)
weights = {
    "residential": 0.10,
    "kindergarten": 0.10,
    "religious": 0.05,
    "hospital": 0.15,
    "cemetries": 0.05,
    "stops": 0.35,
    "exits": 0.25,
}

In [ ]:
# Calculate total score as weighted sum of individual scores
gdf_scores["total_score"] = 0.0

for key, w in weights.items():
    gdf_scores["total_score"] += w * gdf_scores["score_" + key]

In [ ]:
# sort by total score (best locations first)
gdf_scores = gdf_scores.sort_values(
    "total_score",
    ascending=False
).reset_index(drop=True)
gdf_scores.head()

# Visualizations of locations

Overview map of suitable areas <br>
- district boundaries <br>
- suitable areas <br>
- top 5 suitable areas highlighted <br>

In [ ]:
'''Visualisation 1'''

# Prepare data (sort top5 by suitability score in descending order)
top5 = gdf_scores.head(5).copy()
top5 = top5.sort_values(by="total_score", ascending=False)  # Sort by suitability score (highest first)
top5 = top5.to_crs(epsg=3857)
gdf_districts_web = gdf_districts.to_crs(epsg=3857)

fig, ax = plt.subplots(figsize=(12, 12))

# Plot districts
gdf_districts_web.boundary.plot(
    ax=ax,
    color="dimgray",
    linewidth=0.6,
    zorder=1
)

# Plot candidate sites
top5.plot(
    ax=ax,
    column="total_score",
    cmap="Greens",   
    legend=True,
    edgecolor="black",
    linewidth=1.2,
    alpha=0.9,
    zorder=3,
    legend_kwds={
        "label": "Suitability score",
        "shrink": 0.7
    }
)

# Annotate rank (offset to upper-left)
offset_x = -850   # meters (Web Mercator)
offset_y = 200

for rank, (_, row) in enumerate(top5.iterrows(), start=1):
    centroid = row.geometry.centroid
    ax.annotate(
        text=f"{rank}",
        xy=(centroid.x + offset_x, centroid.y + offset_y),
        ha="center",
        va="center",
        fontsize=12,
        fontweight="bold",
        color="black",
        bbox=dict(
            boxstyle="round,pad=0.25",
            fc="white",
            ec="black",
            lw=0.8,
            alpha=0.9
        ),
        zorder=4
    )

ctx.add_basemap(
    ax,
    source=ctx.providers.CartoDB.Positron,
    zoom=12,
    attribution_size=8
)

# Scale bar
scalebar = ScaleBar(
    dx=1,
    units="m",
    location="lower left",
    frameon=True,
    pad=0.6,
    border_pad=0.6
)
ax.add_artist(scalebar)

# Title
ax.set_title(
    "Top 5 Candidate Sites for Stadium Location",
    fontsize=18,
    fontweight="bold",
    pad=18
)

# Subtitle with background box
ax.text(
    0.5, 0.9865,
    "Ranked by composite suitability score",
    transform=ax.transAxes,
    ha="center",
    va="center",
    fontsize=14,
    color="white",
    bbox=dict(
        boxstyle="round,pad=0.2",
        fc="black",
        ec="none",
        alpha=1
    ),
    zorder=5
)

ax.axis("off")
plt.tight_layout()
plt.show()


Interactive map with Kepler.gl

!!! some laptops might have issues to render kepler.gl maps !!! <br>
!!! try to save the map as html and open it in a browser if issues occur !!! <br>
!!! otherwise skip kepler.gl visualization and use folium or static maps only !!!

In [ ]:
'''Visualisation 2'''

# Drop everything except scores and area
top5_kepler = top5.drop(columns=['dist_residential', 'dist_kindergarten', 'dist_religious', 'dist_hospital', 'dist_cemetries', 'dist_stops', 'dist_exits'])

# covert to wgs84 for kepler
top5_WGS84 = top5_kepler.to_crs(epsg=4326)

# to GeoJSON
top5_WGS84_json = top5_WGS84.to_json()  # Top 5 Candidate sites in GeoJSON format

# map frame
map_ = keplergl.KeplerGl(height=600)

# add data
map_.add_data(data=top5_WGS84_json, name="Top 5 Candidate Sites")

#save to html
map_.save_to_html(file_name="kepler_top5_candidate_sites.html")

# show map
map_


'''Comparison
Click on the arrow on the left top
next, click on the layer "Top 5 Candidate Sites" to open the layer settings
Click at the three dots at "fill color"
Color based on: "select on field"
now you can choose the score and compare the top 5 sites based on different scores
'''


barplots of the top 5 suitable areas with their scores per criterion

In [ ]:
'''Visualisation 3'''

import matplotlib.pyplot as plt
import numpy as np

# extract score columns
score_columns = [col for col in top5.columns if col.startswith('score_')]

# create a DataFrame with these columns
top5_scores = top5[score_columns].copy()

# create a Site column to name the sites
top5_scores['Site'] = [f'Site {i+1}' for i in range(len(top5_scores))]

# Plotting
plt.figure(figsize=(14, 8))

# Position fixes
x = np.arange(len(top5_scores)) * 2  # increase spacing between sites
width = 0.20  

# Barplot
for i, column in enumerate(score_columns):
    plt.bar(x + i * width - width / 2, top5_scores[column], width, label=column.replace('score_', '').capitalize())

# Axis and title
plt.xlabel('Top 5 Sites')
plt.ylabel('Scores')
plt.title('Comparison of Various Scores for Top 5 Sites')

plt.xticks(x + width * (len(score_columns) - 1) / 2, [f'Site {i+1}' for i in range(len(top5_scores))])  # Center the side labels

# add legend
plt.legend(title='Scores')

plt.tight_layout()

plt.show()


In [ ]:
print(suitable_area.columns)

barplot comparison of different distances to stops and exits for the top 5 suitable areas

In [ ]:
# extract column
dist_columns = ['dist_exits', 'dist_stops']

# copy dataframe
top5_distances = top5[dist_columns].copy()

# Site Column
top5_distances['Site'] = [f'Site {i+1}' for i in range(len(top5_distances))]

# Plotting
plt.figure(figsize=(12, 6))

# Positioning
x = np.arange(len(top5_distances)) * 2  # Distance between sites
width = 0.35  # Bars Width

# create barplot
for i, column in enumerate(dist_columns):
    plt.bar(x + i * width - width / 2, top5_distances[column], width, label=column.replace('dist_', '').capitalize())

# Axis and title
plt.xlabel('Top 5 Sites')
plt.ylabel('Distance (meters)')
plt.title('Comparison of Distances (Exits and Stops) for Top 5 Sites')

# x-axis ticks
plt.xticks(x + width * (len(dist_columns) - 1) / 2, [f'Site {i+1}' for i in range(len(top5_distances))])  # Center the site labels

# add legend
plt.legend(title='Distances')

plt.tight_layout()

# Plotting
plt.show()
